# Lab 04: Multi-Agent Orchestration for Media Operations

## Business Objective

This notebook demonstrates a production-ready multi-agent orchestration system for automated media operations. Building on the specialized agents from previous labs, we now create an orchestrator that intelligently routes requests to the appropriate agents and coordinates their work to deliver compliant metadata.

### The Challenge

Media operations teams face diverse query types:
- **Simple questions** 
  - about video content ("What teams are playing?")
  - Compliance inquiries ("What are the requirements for social media?")
  - Quality validation ("Is this metadata compliant?")
- **Complex questions**
  - Metadata generation ("Create compliant metadata for this video")

Each query type requires different agents and workflows. Manual routing is inefficient and error-prone.

### The Solution

An intelligent orchestrator agent that:
1. **Analyzes** the query type
2. **Routes** to appropriate specialized agents
3. **Coordinates** multi-step workflows
4. **Validates** outputs against compliance requirements

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR AGENT                           │
│                  (Query Analysis & Routing)                     │
└────────────────────────┬────────────────────────────────────────┘
                         │
         ┌───────────────┼───────────────┬────────────────┐
         │               │               │                │
         ▼               ▼               ▼                ▼
┌─────────────┐  ┌─────────────┐  ┌──────────┐  ┌──────────────┐
│   SPORTS    │  │    NEWS     │  │   FILM   │  │ REQUIREMENTS │
│   AGENT     │  │   AGENT     │  │  AGENT   │  │    AGENT     │
│             │  │             │  │          │  │              │
└──────┬──────┘  └─────────────┘  └──────────┘  └──────┬───────┘
       │                                               │
       │ ┌───────────────────────────────────────────┐ │
       └─┤        REMOTE MCP TOOLS                   ├─┘
         │  (via Bedrock AgentCore Gateway)          │
         ├───────────────────────────────────────────┤
         │ • retrieve_match_info                     │
         │ • lookup_player_info                      │
         │ • retrieve_compliance_requirements        │
         │ • search_knowledge_base                   │
         │ • query_dynamodb                          │
         └───────────────────────────────────────────┘
```

### Agent Types

**Local Agents (Python Functions):**
- Sports Agent - Extracts sports-specific information
- News Agent - Extracts news content (WHO, WHAT, WHEN, WHERE, WHY)
- Film Agent - Extracts film/movie information
- Requirements Agent - Retrieves compliance rules and schemas
- Metadata Agent - Generates compliant metadata
- QC Agent - Validates metadata compliance

**Remote Tools (MCP via Bedrock AgentCore):**
- `retrieve_match_info` - Search match database
- `lookup_player_info` - Query player information
- `retrieve_compliance_requirements` - Get compliance rules
- `search_knowledge_base` - Semantic search across knowledge bases
- `query_dynamodb` - Direct database queries

### Workflow Types

1. **Simple Query**: Orchestrator → Content Agent → Response
2. **Compliance Query**: Orchestrator → Requirements Agent → Response
3. **Metadata Generation**: Orchestrator → Content Agent → Requirements Agent → Metadata Agent → Response
4. **Validation**: Orchestrator → Requirements Agent → QC Agent → Response

## Step 1: Environment Setup

Import required libraries and set up paths for the multi-agent orchestration system.

In [ ]:
import sys
import json
import asyncio
from pathlib import Path
from IPython.display import display, Markdown

# Add helper path
sys.path.insert(0, str(Path.cwd().parent / 'helper'))
from display_helper import print_success, print_info, print_result
from agentcore_helper import AgentCoreHelper

# Add agents path
sys.path.insert(0, str(Path.cwd() / 'agents'))

print_info("Environment setup complete")

## Step 2: Load Shared Resources

Load the shared resources created in the prerequisites notebook using Jupyter's `%store` magic. These resources are used by all agents in the orchestration system.

**Resources Used:**
- **4 Knowledge Bases** - Sports, compliance, news, films
- **2 DynamoDB Tables** - Players, cast members
- **1 AgentCore Gateway** - MCP tools for remote access
- **1 Lambda Function** - 7 tools for agent operations

**Prerequisites:** Run `../../00-prerequisites.ipynb` first to create resources and store variables.

In [ ]:
# Load shared resources from Jupyter store (set by prerequisites notebook)
%store -r sports_kb_id
%store -r compliance_kb_id
%store -r news_kb_id
%store -r films_kb_id
%store -r players_table
%store -r cast_table
%store -r gateway_url
%store -r gateway_id
%store -r AWS_REGION

## Step 3: Verify Resources

Verify that the shared resources are accessible and working correctly.

In [ ]:
import boto3

# Verify Gateway is accessible
print_success(f"✓ Gateway URL: {gateway_url}")
print_success(f"✓ Gateway ID: {gateway_id}")

# Verify DynamoDB tables exist
dynamodb = boto3.client('dynamodb', region_name=AWS_REGION)
try:
    # Check players table
    dynamodb.describe_table(TableName=players_table)
    print_success(f"✓ Players Table: {players_table}")
    
    # Check cast table
    dynamodb.describe_table(TableName=cast_table)
    print_success(f"✓ Cast Table: {cast_table}")
except Exception as e:
    print_info(f"Could not verify tables: {e}")

# Verify Knowledge Bases
bedrock_agent = boto3.client('bedrock-agent', region_name=AWS_REGION)
try:
    kb_count = 0
    for kb_id in [sports_kb_id, compliance_kb_id, news_kb_id, films_kb_id]:
        bedrock_agent.get_knowledge_base(knowledgeBaseId=kb_id)
        kb_count += 1
    print_success(f"✓ All {kb_count} Knowledge Bases verified")
except Exception as e:
    print_info(f"Could not verify Knowledge Bases: {e}")

print_success("Resource verification complete!")

## Step 4: Understanding the Orchestrator Agent

The orchestrator agent is the central coordinator that intelligently routes requests and manages multi-agent workflows.

**Core Responsibilities:**

**1. Query Type Detection**

The orchestrator uses LLM reasoning to classify incoming requests:

```python
QUESTION about video content → Route to extraction agent → Return concise answer
QUESTION about compliance → Route to requirements agent → List requirements
VALIDATE metadata → Requirements + QC agents → Report Pass/Fail with issues
GENERATE metadata → Full multi-agent workflow → Return structured JSON
```

**How Detection Works:**
- Analyzes task_description using Claude
- Identifies intent (question, generate, validate)
- Determines content type (sports, news, film)
- Selects appropriate workflow path

**2. Tool Selection**

Based on query type, the orchestrator has access to these local agent tools:

- **extract_sports_content** - Analyzes sports videos, matches with database
- **extract_news_content** - Extracts WHO, WHAT, WHEN, WHERE, WHY
- **extract_film_content** - Identifies movies, actors, scenes
- **get_compliance_requirements** - Retrieves rules from Knowledge Base
- **get_output_schema** - Gets JSON schema for metadata format
- **generate_metadata** - Creates compliant metadata from inputs
- **validate_metadata** - QC checks against requirements
- **enhance_metadata** - Improves existing metadata

**3. Workflow Coordination**

For complex requests, the orchestrator chains multiple agents in sequence:

```
Generate Metadata Workflow:
┌─────────────────────────────────────────────────────────┐
│ 1. Extract Content (Sports/News/Film Agent)            │
│    → Calls MCP tools: retrieve_match_info,             │
│      lookup_player_info via AgentCore Gateway          │
│    → Returns: teams, players, score, key moments       │
├─────────────────────────────────────────────────────────┤
│ 2. Get Requirements (Requirements Agent)               │
│    → Calls MCP tool: search_knowledge_base             │
│    → Returns: compliance rules, character limits       │
├─────────────────────────────────────────────────────────┤
│ 3. Get Schema (Requirements Agent)                     │
│    → Calls MCP tool: search_knowledge_base             │
│    → Returns: JSON schema, required fields             │
├─────────────────────────────────────────────────────────┤
│ 4. Generate Metadata (Metadata Agent)                  │
│    → Combines: content + requirements + schema         │
│    → Returns: compliant JSON metadata                  │
├─────────────────────────────────────────────────────────┤
│ 5. Validate Output (QC Agent) [Optional]               │
│    → Checks compliance, returns Pass/Fail              │
└─────────────────────────────────────────────────────────┘
```

**Key Design Patterns:**
- **Separation of Concerns** - Each agent has a specific domain
- **Composability** - Agents can be combined in different workflows
- **Extensibility** - New agents can be added without changing orchestrator
- **Error Handling** - Orchestrator manages failures and retries

**Let's examine the orchestrator agent code:**

In [ ]:
with open("agents/orchestrator_agent.py", "r") as f:
    code = f.read()

display(Markdown(f"""```python
{code}
```"""))


## Step 5: Test the Orchestrator Agent Locally

Test the orchestrator locally before deploying to AgentCore Runtime.

**Test Scenarios:**
1. Simple question about video content
2. Full metadata generation workflow

In [ ]:
# Import the orchestrator
from orchestrator_agent import process_video_metadata

# Sample video summary
video_summary = """In a professional basketball arena, a high-stakes game unfolds between two teams 
in red and white uniforms. The action centers on a player in red (number 14) who drives to the basket 
and attempts a powerful dunk. The opposing team in white (number 8) tries to block the shot. The 
scoreboard shows the red team leading 37-23. The dunk attempt is contested by multiple players from 
both teams. The crowd cheers loudly as the play unfolds. The red team player successfully completes 
the dunk, increasing their lead. The opposing team regroups, with players in white (numbers 14 and 8) 
preparing to continue the game. The scene captures the intensity and excitement of professional 
basketball, with players demonstrating athleticism and competitive spirit."""

print_success("Orchestrator loaded successfully!")

### Test 1: Simple Query - "What teams are playing?"

The orchestrator detects this is a simple question, routes to the Sports Agent, which uses MCP tools to match video details against the database.

**Flow:** Orchestrator → Sports Agent → retrieve_match_info → Response

In [ ]:
async def test_simple_query():
    payload = {
        "video_summaries": [video_summary],
        "s3_url": "",
        "task_description": "What teams are playing in this video?"
    }
    
    print_info("Query: What teams are playing in this video?")
    print_info("Expected: Orchestrator → Sports Agent → Answer")
    
    response_chunks = []
    async for chunk in process_video_metadata(payload):
        response_chunks.append(chunk)
    
    result = ''.join(response_chunks)
    print_result("RESULT", result)
    return result

# Run the test
result1 = await test_simple_query()

### Test 2: Metadata Generation - "Generate compliant metadata"

Tests the full multi-agent workflow: extract content → get requirements → get schema → generate metadata.

**Flow:** Orchestrator → Sports Agent → Requirements Agent → Metadata Agent → JSON Response

In [ ]:
async def test_metadata_generation():
    payload = {
        "video_summaries": [video_summary],
        "s3_url": "",
        "task_description": "Generate compliant social media metadata for this video"
    }
    
    print_info("Query: Generate compliant social media metadata for this video")
    
    response_chunks = []
    async for chunk in process_video_metadata(payload):
        response_chunks.append(chunk)
    
    result = ''.join(response_chunks)

    print_result("RESULT", result)
    
    return result

# Run the test
result2 = await test_metadata_generation()

## Step 6: Deploy Orchestrator to AgentCore Runtime

Now let's deploy our orchestrator agent to Amazon Bedrock AgentCore Runtime for production use. This enables:

### Benefits of AgentCore Deployment
* **Scalability**: Automatic scaling based on demand
* **Managed Infrastructure**: No server management required
* **Streaming Support**: Real-time response streaming
* **Security**: Built-in IAM integration and encryption
* **Monitoring**: CloudWatch integration for metrics and logs

### Deployment Architecture

```
┌─────────────────────────────────────────────────────────┐
│                    Client Application                   │
└────────────────────────┬────────────────────────────────┘
                         │ HTTPS/SSE
                         ▼
┌─────────────────────────────────────────────────────────┐
│          Amazon Bedrock AgentCore Runtime               │
│  ┌───────────────────────────────────────────────────┐  │
│  │         Orchestrator Agent (Container)            │  │
│  │  ┌─────────────────────────────────────────────┐  │  │
│  │  │  • Sports Agent                             │  │  │
│  │  │  • News Agent                               │  │  │
│  │  │  • Film Agent                               │  │  │
│  │  │  • Requirements Agent                       │  │  │
│  │  │  • Metadata Agent                           │  │  │
│  │  │  • QC Agent                                 │  │  │
│  │  └─────────────────────────────────────────────┘  │  │
│  └───────────────────────────────────────────────────┘  │
└────────────────────────┬────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────┐
│              AWS Services & Resources                   │
│  • Amazon Bedrock (LLM)                                 │
│  • AgentCore Gateway (MCP Tools)                        │
│  • Knowledge Bases                                      │
│  • DynamoDB Tables                                      │
└─────────────────────────────────────────────────────────┘
```

### Configure AgentCore Runtime Deployment

Configure the orchestrator for deployment to AgentCore Runtime using the Bedrock AgentCore Starter Toolkit.

**What is AgentCore Runtime?**
- Managed container service for running agents in production
- Automatically scales based on request volume
- Provides built-in monitoring, logging, and health checks
- Supports streaming responses and session management

**Configuration Steps:**

**1. Create Execution Role** - IAM role with required permissions:
- **SSM GetParameter** - Read gateway URL from Parameter Store
- **Bedrock InvokeModel** - Call foundation models (Claude, etc.)
- **CloudWatch Logs** - Write runtime logs and metrics
- **ECR Access** - Pull container images
- **Gateway Access** - Invoke AgentCore Gateway for MCP tools

**2. Configure Runtime** - Specify deployment parameters:
- **Entrypoint** - Python file with `BedrockAgentCoreApp` instance
- **Execution Role** - IAM role ARN created above
- **Auto-create ECR** - Automatically create ECR repository if needed
- **Requirements** - Python dependencies to install in container
- **Agent Name** - Unique identifier for the runtime

**What Gets Generated:**
- `Dockerfile` - Container build instructions with Python base image
- `.bedrock_agentcore.yaml` - Runtime configuration file
- Build artifacts in `.bedrock_agentcore/` directory

**Reference:** [AgentCore Runtime Configuration](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-runtime-config.html)

In [ ]:
!rm -rf helper && cp -R ../helper helper

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

print_info(f"Configuring AgentCore Runtime in region: {region}")

# Initialize AgentCore helper and runtime
agentcore_helper = AgentCoreHelper(region_name=region)
agentcore_runtime = Runtime()

# Create execution role with required permissions
# This role allows the runtime to:
# - Read gateway URL from SSM Parameter Store
# - Invoke Bedrock models for LLM inference
# - Write logs to CloudWatch
# - Pull container images from ECR

agent_name = "orchestrator_agent"
agent_execution_role_name = f"{agent_name}_role"
agent_execution_policy_name = f"{agent_name}_policy"

execution_role_arn = agentcore_helper.create_agentcore_runtime_execution_role(
    role_name=agent_execution_role_name,
    policy_name=agent_execution_policy_name
)

# Configure the runtime deployment
response = agentcore_runtime.configure(
    entrypoint="agentcore_orchestrator_agent.py",  # File with BedrockAgentCoreApp
    execution_role=execution_role_arn,              # IAM role for runtime
    auto_create_ecr=True,                           # Create ECR repo automatically
    requirements_file="requirements.txt",           # Python dependencies
    region=region,                                  # AWS region
    agent_name=agent_name                           # Unique runtime name
)

print_success("AgentCore Runtime configured successfully!")
print_info(f"Agent Name: orchestrator_agent")
print_info(f"Entrypoint: agentcore_orchestrator_agent.py")
print_info(f"Execution Role: {execution_role_arn}")

### Launch Orchestrator to AgentCore Runtime

Deploy the orchestrator container to AgentCore Runtime for production use.

**Launch Process:**

**1. Build Docker Image** - Creates container with orchestrator code:
- Uses generated Dockerfile with Python 3.11 base image
- Installs Python dependencies from requirements.txt
- Packages all agent code and helper modules
- Sets up proper entrypoint for AgentCore Runtime
- Configures environment variables for AWS SDK

**2. Push to ECR** - Uploads container to Amazon Elastic Container Registry:
- Creates ECR repository if it doesn't exist
- Authenticates with ECR using AWS credentials
- Tags image with version information
- Pushes to private ECR registry in your account

**3. Create Runtime** - Provisions AgentCore Runtime:
- Deploys container to managed ECS infrastructure
- Configures auto-scaling policies (min/max instances)
- Sets up networking and security groups
- Enables CloudWatch monitoring and logging
- Creates HTTPS endpoint for invocation

**4. Runtime States** - Monitor deployment progress:
- `CREATING` - Runtime is being provisioned
- `READY` - Runtime is ready to accept requests
- `CREATE_FAILED` - Deployment failed (check CloudWatch logs)
- `UPDATING` - Runtime is being updated

**Note:** This process typically takes 5-10 minutes. Use `auto_update_on_conflict=True` to update existing runtime with same name.

**Reference:** [AgentCore Runtime Deployment](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-runtime-deploy.html)

In [ ]:
print_info("Launching orchestrator to AgentCore Runtime... This may take 5-10 minutes.")
print_info("Steps: Build Docker → Push to ECR → Create Runtime → Configure Networking")

# Launch the runtime
# auto_update_on_conflict=True will update existing runtime if name conflicts
launch_result = agentcore_runtime.launch(
    auto_update_on_conflict=True
)

print_success("Orchestrator launched successfully!")
print_info(f"Agent ARN: {launch_result.agent_arn}")
print_info(f"Agent ID: {launch_result.agent_id}")
print_info(f"ECR URI: {launch_result.ecr_uri}")
print_info("\nThe runtime is now being provisioned. Check status in next cell.")

### Check Deployment Status

Monitor the deployment status until the runtime reaches `READY` state.

In [ ]:
import time

print_info("Checking deployment status...")

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print_info(f"Current status: {status}")

if status == 'READY':
    print_success(f"✅ AgentCore Runtime is READY!")
else:
    print(f"❌ Deployment failed with status: {status}")

## Step 7: Test the Deployed Orchestrator

Test the deployed orchestrator using the AgentCore Runtime endpoint.

**How It Works:**
1. Client sends request via boto3 to runtime endpoint
2. Runtime forwards to container and invokes orchestrator
3. Orchestrator analyzes query, routes to agents, coordinates workflow
4. Response streams back as Server-Sent Events (SSE)

**Test Cases:**
1. Simple query about video content
2. Full metadata generation workflow
3. Metadata validation check

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client('bedrock-agentcore', region_name=region)

print_info(f"Ready to invoke agent: {agent_arn}")

### Test 1: Simple Query via AgentCore Runtime

Ask a simple question about the video content. The response streams back in real-time.

In [ ]:
test_payload = {
    "video_summaries": [video_summary],
    "s3_url": "",
    "task_description": "Who are the key figures from this video?"
}

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps(test_payload)
)

# Handle streaming response (SSE format)
if "text/event-stream" in boto3_response.get("contentType", ""):
    print_info("Processing streaming response...")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')
                content.append(data)
    
    full_response = " ".join(content)
    print_result("FINAL RESULT", full_response)
else:
    print_info("Non-streaming response received")
    print(boto3_response)

### Test 2: Metadata Generation via AgentCore Runtime

Test the full workflow: Sports Agent extracts content → Requirements Agent gets rules → Metadata Agent generates compliant JSON.

In [ ]:
payload = {
    "video_summaries": [video_summary],
    "s3_url": "",
    "task_description": "Generate compliant social media metadata for this video"
}

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps(payload)
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    print_info("Processing streaming response...")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')
                content.append(data)
    
    full_response = " ".join(content)
    print_result("FINAL RESULT", full_response)
else:
    print_info("Non-streaming response received")
    print(boto3_response)

### Test 3: Validation Query - Evaluate Metadata Compliance

Test the QC workflow: Requirements Agent gets rules → QC Agent validates metadata → Returns Pass/Fail report.

In [ ]:
import boto3
import json

payload = {
    "video_summaries": [video_summary],
    "s3_url": "",
    "task_description": "Evaluate if this metadata for a sports video is compliant for social media",
    "existing_metadata": '{"description": "🔥 POSTER ALERT! @MetropolitanKnights #14 Thompson throws down a MONSTER dunk over two @AlpineWolves defenders in the PBF showdown at Avalon Center! This electrifying play sparked a 16-2 run as the Knights dominate 37-23."}'
}

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps(payload)
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    print_info("Processing streaming response...")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')
                content.append(data)
    
    full_response = " ".join(content)
    print_result("VALIDATION RESULT", full_response)
else:
    print_info("Non-streaming response received")
    print(boto3_response)

# 🎉 Congratulations!

You have successfully:

## ✅ Built a Multi-Agent Orchestration System
- Created specialized agents for sports, news, and film content
- Implemented compliance checking and metadata generation
- Coordinated complex multi-step workflows

## ✅ Deployed to Production
- Containerized the orchestrator for AgentCore Runtime
- Deployed to a managed, scalable infrastructure
- Tested with streaming responses

## ✅ Integrated AWS Services
- Amazon Bedrock for LLM capabilities
- AgentCore Gateway for remote MCP tools
- Knowledge Bases for semantic search
- DynamoDB for structured data

## What You've Learned

1. **Multi-Agent Architecture**: How to design and implement agent orchestration
2. **Query Type Detection**: Intelligent routing based on request analysis
3. **Workflow Coordination**: Chaining multiple agents for complex tasks
4. **AgentCore Deployment**: Production deployment with managed infrastructure
5. **Streaming Responses**: Real-time result delivery for better UX
6. **AWS Integration**: Leveraging multiple AWS services together

## Resources

- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [AgentCore Runtime Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore.html)
- [Strands Agents Documentation](https://github.com/strands-agents/sdk-python)
- [MCP Protocol Specification](https://modelcontextprotocol.io/)

---

<div style="padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 10px; color: white; text-align: center; margin: 20px 0;">
    <h2 style="margin: 0; color: white;">🚀 Ready for Production!</h2>
    <p style="margin: 10px 0 0 0; font-size: 18px;">Your multi-agent orchestration system is now deployed and ready to handle real-world media operations at scale!</p>
</div>